## ``torch.autograd``による自動微分法
ニューラルネットワークを学習する際、最も頻繁に使用されるアルゴリズムが逆伝播です。このアルゴリズムでは、パラメータ（モデルの重み）は、与えられたパラメータに関する損失関数の勾配に従って調整されます。損失関数は、ニューラルネットワークが生成する期待出力と実際の出力の差を計算する。目標は、損失関数の結果を可能な限りゼロに近づけることです。アルゴリズムは、ネットワーク網を後方から走査して重みとバイアスを調整し、モデルを再学習させます。これが逆伝播と呼ばれる所以です。このように、損失を0に近づけるために時間をかけてモデルを再学習させるバック＆フォワードのプロセスを、勾配降下と呼びます。

その勾配を計算するために、PyTorchにはtorch.autogradという微分エンジンが組み込まれています。これは、任意の計算グラフに対する勾配の自動計算をサポートします。


In [2]:
import torch

x = torch.ones(5)   # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5,3, requires_grad=True)    # weight
b = torch.randn(3, requires_grad=True)      # bias
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x000001D0EF52D1E0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x000001D0EF52E860>


## 勾配の計算 Computing gradients
ニューラルネットワークのパラメーターの重みを最適化するために、パラメーターに関する損失関数の導関数を計算する必要がある。これらの導関数を計算するために、loss.backward()を呼び出し、w.gradとb.gradから値を取得する

requires_gradプロパティがTrueに設定されているノードに対してのみ、gradプロパティを取得できます。パフォーマンス上の理由から、backwardを使った勾配計算は、与えられたグラフに対して一度しか実行できません。同じグラフで何度もbackwardを呼び出す必要がある場合は、backward呼び出しにretain_graph=Trueを渡す必要があります。

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1290, 0.0885, 0.0061],
        [0.1290, 0.0885, 0.0061],
        [0.1290, 0.0885, 0.0061],
        [0.1290, 0.0885, 0.0061],
        [0.1290, 0.0885, 0.0061]])
tensor([0.1290, 0.0885, 0.0061])


## 勾配追跡の無効
デフォルトでは、requires_grad=Trueのテンソルは、計算履歴を追跡し、勾配計算をサポートします。しかし、その必要がない場合もあります。例えば、モデルを訓練して、それを入力データに適用したい場合、つまり、ネットワークを通して前方計算を行いたいだけです。計算コードを `torch.no_grad()` ブロックで囲む、`deatach()`メソッドでトラッキング計算を停止することができます：

In [4]:
print(z.requires_grad)

# z_det = z.detach()

with torch.no_grad():
    z = torch.matmul(x, w) + b
    
print(z.requires_grad)

True
False


### 勾配追跡を無効にしたい理由
- ニューラルネットワークのいくつかのパラメータを凍結したパラメータとしてマークするためです。これは事前に訓練されたネットワークの微調整を行う際に非常によくあるシナリオです。
- 勾配を追跡しないテンソルでの計算がより効率的であるため、フォワードパスのみを行う場合に計算を高速化するため。

In [5]:
inp = torch.eye(5, requires_grad=True)
out = (inp + 1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
